# Sequential Importance Sampling (SIS)

heres an algorithm from a big book, Probabilistic Machine Learning Advanced Topics:

![image](images/Screenshot%20from%202024-10-17%2013-54-06.png)

Actually implement simons alg for static tracking first as it will be easier:

![images](images/GreenMaskAlg1.png)
![images](images/GreenMaskAlg2.png)

In [ ]:
%% capture output
%pip install numpy
%pip install matplotlib

In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [119]:
def initial_theta(n):
    return np.random.randn(n)+2

def normal_dist(x,mu,sigma):
    return (1/(sigma*(np.pi*2)**1/2))*np.e**(-0.5*((x-mu)/sigma)**2)

def target_dist_p(x):
    return normal_dist(x,5,0.5)

def new_proposal(old_thetas):
    return old_thetas+np.random.randn(len(old_thetas))

def initial_weights(theta):
    weights = []
    for t in theta:
        weights += [normal_dist(t,0,1)]
    return weights

def resample(t:list[float]):
    n = len(t)
    mu = sum(t)/n
    return np.random.randn(n)+mu

def norm_weights(w_un):
    w_sum = sum(w_un)
    weights = []
    for w in w_un:
        weights += [w/w_sum]
    print(np.array(weights))
    return weights

def get_neff(weights):
    squared_sum = 0
    for w in weights:
        squared_sum += w*w
    return 1/squared_sum

n = 10
k = 1
# the parallel samples are 1 to N in the paper
# will be a k by n matrix by the end of the for loop
theta = []
weights_unnormed = []
weights_normed = []

theta += [initial_theta(n)]
weights_unnormed = initial_weights(theta)

for i in range(100):
    weights_normed += [norm_weights(weights_unnormed[-1])]
    Neff = get_neff(weights_normed[-1])
    if Neff<n/2:
        theta[-1] = [resample(theta[-1])]
        weights_normed = np.array(weights_normed)
        print(weights_normed)
        weights_unnormed[-1] = [1]*len(weights_normed[-1])
        print((weights_unnormed))
    k+=1
    theta+=[new_proposal(theta[-1])]
    new_unnormed_weights = []
    for w,t_old,t_new in zip(weights_unnormed[-1],theta[-2],theta[-1]):
        new_unnormed_weights += [w*(target_dist_p(t_new)/target_dist_p(t_old))]
    weights_unnormed += [new_unnormed_weights]

theta_pred = [sum(t) for t in theta]

plt.plot(theta_pred)



[0.0190838  0.44057031 0.01072931 0.00366    0.04640961 0.01033663
 0.37340956 0.00052974 0.09268698 0.00258406]
[[0.0190838  0.44057031 0.01072931 0.00366    0.04640961 0.01033663
  0.37340956 0.00052974 0.09268698 0.00258406]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


ValueError: non-broadcastable output operand with shape (1,10) doesn't match the broadcast shape (1,1,10)